<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/218_Cross_Sell_Upsell_Orchestrator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Business Logic Explained: Cross-Sell & Upsell Orchestrator

**Purpose:** Walkthrough of business logic design decisions, alternatives, and trade-offs

---

## 🎯 Overview: What the Business Logic Does

The business logic orchestrator identifies **three types of opportunities**:

1. **Routine Gaps** - Missing essential skincare steps (cleanser, toner, serum, moisturizer, SPF)
2. **Cross-Sell Opportunities** - Complementary products based on what customer owns
3. **Upsell Opportunities** - Replenishment needs and bundle opportunities

Then it **scores and ranks** all opportunities to prioritize the highest-value recommendations.

---

## 📊 Part 1: Routine Gap Detection

### What It Does

```python
identify_routine_gaps(customer_categories, essential_categories)
```

**Example:**
- Customer has: `["cleanser", "moisturizer"]`
- Essential categories: `["cleanser", "toner", "serum", "moisturizer", "spf"]`
- Returns: `["toner", "serum", "spf"]` (missing categories)

### Why This Approach?

**✅ Simple Set Difference**
- Fast, deterministic
- Easy to understand and debug
- No complex logic needed

**Alternative Approaches We Could Have Used:**

1. **LLM-Based Gap Detection**
   ```python
   # Alternative: Use LLM to analyze customer routine
   prompt = f"Customer has {customer_categories}. What's missing?"
   gaps = llm_call(prompt)
   ```
   - ❌ **Why not:** Expensive, slow, non-deterministic
   - ✅ **When to use:** If routine logic is complex or changes frequently

2. **Rule-Based with Priorities**
   ```python
   # Alternative: Weighted importance
   essential = {"cleanser": 1.0, "toner": 0.8, "serum": 0.9, ...}
   gaps = sorted([cat for cat in essential if cat not in customer_categories],
                 key=lambda x: essential[x], reverse=True)
   ```
   - ✅ **Why not chosen:** Current approach is simpler, priorities handled in scoring
   - ✅ **When to use:** If you need to prioritize which gaps to fill first

3. **Machine Learning Model**
   ```python
   # Alternative: Train model on customer behavior
   gaps = ml_model.predict_missing_products(customer_features)
   ```
   - ❌ **Why not:** Requires training data, overkill for MVP
   - ✅ **When to use:** With large dataset and complex patterns

**Decision:** Simple set difference because:
- Fast and reliable
- Business rules are clear (5 essential categories)
- Can add complexity later if needed

---

## 🔄 Part 2: Replenishment Detection

### What It Does

```python
check_replenishment_needs(customer_products, product_catalog)
```

**Logic:**
1. For each product customer owns, get purchase date
2. Calculate days since purchase
3. Compare to product's `replenishment_cycle_days`
4. Flag if `days_until_replenishment <= 0` (due) or `<= 7` (approaching)

**Example:**
- Customer bought P001 (cleanser) on 2024-01-01
- Today is 2024-02-15 (45 days later)
- Cleanser replenishment cycle: 30 days
- Result: `replenishment_due: True` (15 days overdue)

### Why This Approach?

**✅ Time-Based Calculation**
- Uses actual purchase dates
- Accounts for product-specific cycles (30-45 days)
- Provides urgency levels (due vs. approaching)

**Alternative Approaches:**

1. **Usage-Based Prediction**
   ```python
   # Alternative: Predict based on usage patterns
   predicted_usage = ml_model.predict_usage(customer_id, product_id)
   days_until_empty = predicted_usage / daily_usage_rate
   ```
   - ❌ **Why not:** Requires usage tracking data we don't have
   - ✅ **When to use:** With IoT devices or usage logging

2. **Fixed Time Windows**
   ```python
   # Alternative: All products = 30 days
   if days_since_purchase > 30:
       replenishment_due = True
   ```
   - ❌ **Why not:** Ignores product differences (SPF vs. mask have different cycles)
   - ✅ **When to use:** If all products have same cycle

3. **Customer Behavior Patterns**
   ```python
   # Alternative: Learn from customer's historical purchase frequency
   avg_days_between_purchases = calculate_customer_avg(customer_id, product_id)
   if days_since_purchase > avg_days_between_purchases:
       replenishment_due = True
   ```
   - ✅ **Why not chosen:** Good idea, but requires historical data
   - ✅ **When to use:** With 6+ months of purchase history

**Decision:** Time-based with product-specific cycles because:
- Uses data we have (purchase dates, replenishment cycles)
- Accounts for product differences
- Simple to understand and maintain

---

## 🛒 Part 3: Cross-Sell Opportunity Detection

### What It Does

```python
find_cross_sell_opportunities(customer_data, product_catalog, routine_gaps)
```

**Two Sources of Opportunities:**

1. **Routine Gap Filling** (Priority 1)
   - Customer missing "toner" → Recommend any toner product
   - Fills essential routine steps

2. **Product-Based Cross-Sells** (Priority 2)
   - Customer owns P001 (cleanser)
   - P001 has `recommended_cross_sells: ["P002", "P003", "P004", "P005"]`
   - Recommend those products (if customer doesn't own them)

**Deduplication:** Uses `recommended_product_ids` set to avoid recommending same product twice.

### Why This Two-Tier Approach?

**✅ Combines Rule-Based + Relationship-Based**
- Routine gaps = business logic (complete the routine)
- Product cross-sells = data-driven (product relationships)

**Alternative Approaches:**

1. **Only Routine Gaps**
   ```python
   # Alternative: Only fill gaps, ignore product relationships
   opportunities = [products_in_category(gap) for gap in routine_gaps]
   ```
   - ❌ **Why not:** Misses valuable product-specific recommendations
   - ✅ **When to use:** If product relationships aren't reliable

2. **Only Product Relationships**
   ```python
   # Alternative: Only use recommended_cross_sells
   for product_owned in customer_products:
       opportunities.extend(product_owned.recommended_cross_sells)
   ```
   - ❌ **Why not:** Might miss essential routine steps
   - ✅ **When to use:** If routine completion isn't a priority

3. **Collaborative Filtering**
   ```python
   # Alternative: "Customers who bought X also bought Y"
   similar_customers = find_similar_customers(customer_id)
   opportunities = products_bought_by_similar_customers(similar_customers)
   ```
   - ❌ **Why not:** Requires large customer base and purchase history
   - ✅ **When to use:** With 1000+ customers and rich purchase data

4. **LLM-Based Recommendations**
   ```python
   # Alternative: LLM analyzes customer profile and suggests products
   prompt = f"Customer profile: {customer_data}. Recommend products."
   opportunities = llm_call(prompt)
   ```
   - ❌ **Why not:** Expensive, non-deterministic, hard to debug
   - ✅ **When to use:** For complex, nuanced recommendations

**Decision:** Two-tier approach (gaps + relationships) because:
- Balances business goals (complete routine) with data (product relationships)
- Fast and deterministic
- Easy to explain to stakeholders

---

## 📈 Part 4: Upsell Opportunity Detection

### What It Does

```python
find_upsell_opportunities(customer_data, replenishment_needs)
```

**Current Implementation:**
- Only handles **replenishment upsells** (products needing refill)
- Creates urgency levels (high if due, medium if approaching)

**Note:** Bundle upsells mentioned in docstring but not implemented (MVP decision).

### Why This Limited Approach?

**✅ Start Simple**
- Replenishment is clear, high-value opportunity
- Bundles require more complex logic (what products to bundle? pricing?)

**Alternative Approaches:**

1. **Bundle Detection** (Not Implemented)
   ```python
   # Alternative: Suggest bundles of missing products
   if len(routine_gaps) >= 2:
       bundle = create_bundle(routine_gaps)
       opportunities.append({
           "type": "bundle",
           "products": bundle,
           "discount": 0.15,  # 15% off bundle
           "rationale": "Complete your routine and save!"
       })
   ```
   - ✅ **Why not implemented:** More complex, requires bundle pricing logic
   - ✅ **When to add:** Phase 2, when we have bundle definitions

2. **Premium Product Upsells**
   ```python
   # Alternative: Suggest premium version of product customer owns
   if customer_owns_basic_version(product_id):
       premium_version = get_premium_version(product_id)
       opportunities.append(premium_version)
   ```
   - ✅ **Why not implemented:** Requires product tier data (basic/premium)
   - ✅ **When to add:** When product catalog has tier information

3. **Quantity Upsells**
   ```python
   # Alternative: "Buy 2, get 1 free" or "Subscribe and save"
   if customer_buys_single(product_id):
       opportunities.append({
           "type": "quantity_upsell",
           "quantity": 2,
           "discount": 0.10
       })
   ```
   - ✅ **Why not implemented:** Requires subscription/promotion logic
   - ✅ **When to add:** When business model supports it

**Decision:** Replenishment-only for MVP because:
- Clear value proposition
- Simple to implement
- High conversion potential
- Can add bundles/premium later

---

## 🎯 Part 5: Opportunity Scoring

### What It Does

```python
score_opportunity(opportunity, customer_data, product, routine_gaps, replenishment_needs)
```

**Four Scoring Dimensions (Weighted):**

1. **Business Value (40%)**
   - Formula: `price × margin_multiplier`
   - Margin multipliers: high=1.5, medium=1.0, low=0.7
   - **Why 40%:** Revenue is primary business goal

2. **Customer Fit (30%)**
   - Price sensitivity match (low sensitivity → higher prices OK)
   - Loyalty tier multiplier (gold=1.2, silver=1.0, bronze=0.8)
   - Churn risk urgency (higher churn = more urgency)
   - **Why 30%:** Customer satisfaction affects retention

3. **Routine Completeness (20%)**
   - Essential category gaps = 15.0 points
   - Routine gap products = 12.0 points
   - Product cross-sells = 8.0 points
   - **Why 20%:** Completing routine increases LTV

4. **Replenishment Urgency (10%)**
   - Past due (>30 days) = 10.0 points
   - Approaching (20-30 days) = 7.0 points
   - **Why 10%:** Important but less critical than other factors

**Final Score:** Weighted sum of all four dimensions

### Why This Scoring System?

**✅ Multi-Objective Optimization**
- Balances revenue (business value) with customer fit (satisfaction)
- Accounts for strategic goals (routine completion, retention)

**Alternative Scoring Approaches:**

1. **Revenue-Only Scoring**
   ```python
   # Alternative: Only consider price × margin
   score = price * margin_multiplier
   ```
   - ❌ **Why not:** Ignores customer fit, might recommend wrong products
   - ✅ **When to use:** If revenue is only goal (short-term focus)

2. **Customer Satisfaction-Only**
   ```python
   # Alternative: Only consider customer fit
   score = customer_fit_score
   ```
   - ❌ **Why not:** Might recommend low-margin products
   - ✅ **When to use:** If retention is only goal

3. **Machine Learning Scoring**
   ```python
   # Alternative: Train model on conversion data
   features = [price, margin, customer_tier, churn_risk, ...]
   score = ml_model.predict_conversion_probability(features)
   ```
   - ✅ **Why not chosen:** Requires training data (conversion history)
   - ✅ **When to use:** With 1000+ conversions and A/B test results

4. **Rule-Based with More Granularity**
   ```python
   # Alternative: More detailed rules
   if customer_tier == "gold" and price_sensitivity == "low":
       score_multiplier = 1.5
   elif customer_tier == "bronze" and price_sensitivity == "high":
       score_multiplier = 0.6
   # ... many more rules
   ```
   - ✅ **Why not chosen:** Current approach is simpler, easier to maintain
   - ✅ **When to use:** If business rules are complex and well-defined

5. **LLM-Based Scoring**
   ```python
   # Alternative: LLM evaluates opportunity
   prompt = f"Score this opportunity: {opportunity} for customer: {customer_data}"
   score = llm_call(prompt)
   ```
   - ❌ **Why not:** Expensive, non-deterministic, can't explain scores
   - ✅ **When to use:** For complex, nuanced scoring that rules can't capture

**Decision:** Weighted multi-dimensional scoring because:
- Balances multiple business objectives
- Transparent and explainable (can show why each opportunity scored high)
- Easy to adjust weights as business priorities change
- Fast and deterministic

### Weight Justification

**Why 40% Business Value?**
- Primary goal: Increase revenue
- Higher weight ensures high-margin products rank well

**Why 30% Customer Fit?**
- Secondary goal: Customer satisfaction
- Prevents recommending expensive products to price-sensitive customers

**Why 20% Routine Completeness?**
- Strategic goal: Build complete routines (increases LTV)
- Lower weight because not all customers want complete routines

**Why 10% Replenishment Urgency?**
- Tactical goal: Timely replenishment
- Lower weight because replenishment is time-sensitive but less strategic

**Could We Adjust Weights?**
- ✅ Yes! This is a design decision, not a technical constraint
- Test different weights with A/B testing
- Adjust based on business priorities

---

## 📊 Part 6: Ranking & Summary

### What It Does

1. **Ranking:** Simple sort by `raw_score` (descending)
2. **Summary:** Calculate metrics (total opportunities, revenue, high-value count)

### Why Simple Ranking?

**✅ Transparent and Fast**
- Easy to understand: highest score = best opportunity
- Fast: O(n log n) sort

**Alternatives:**

1. **Multi-Criteria Ranking**
   ```python
   # Alternative: Rank by multiple criteria
   ranked = sorted(opportunities,
                   key=lambda x: (x.business_value, x.customer_fit, x.routine_score),
                   reverse=True)
   ```
   - ✅ **Why not chosen:** Final score already combines criteria
   - ✅ **When to use:** If you want to show rankings by different dimensions

2. **Diversity Ranking**
   ```python
   # Alternative: Ensure variety (not all same category)
   ranked = diversity_rank(opportunities, max_per_category=2)
   ```
   - ✅ **Why not chosen:** MVP focuses on best opportunities, not diversity
   - ✅ **When to use:** When showing multiple recommendations to customer

3. **Personalized Ranking**
   ```python
   # Alternative: Adjust ranking based on customer preferences
   ranked = personalized_rank(opportunities, customer_preferences)
   ```
   - ✅ **Why not chosen:** Scoring already includes customer fit
   - ✅ **When to use:** If customers have explicit preferences

**Decision:** Simple score-based ranking because:
- Scoring already accounts for all factors
- Simple to understand and debug
- Fast and efficient

---

## 🎓 Key Design Principles

### 1. **Rule-Based First, LLM Later**
- ✅ Current: Deterministic rules
- ✅ Future: Add LLMs for fuzzy matching, natural language rationale

### 2. **Transparency Over Complexity**
- ✅ Scores are explainable (can show breakdown)
- ✅ Logic is readable and maintainable

### 3. **MVP → Enhanced**
- ✅ Start simple (replenishment only)
- ✅ Add complexity later (bundles, premium upsells)

### 4. **Separate Concerns**
- ✅ Gap detection = separate function
- ✅ Cross-sell = separate function
- ✅ Scoring = separate function
- ✅ Easy to test and modify independently

---

## 🔄 What We Could Change

### Easy Changes (No Architecture Changes)

1. **Adjust Scoring Weights**
   - Change 40/30/20/10 to different ratios
   - Test impact on recommendations

2. **Add More Replenishment Urgency Levels**
   - Currently: due, approaching
   - Could add: critical (60+ days), warning (14-7 days)

3. **Prioritize Certain Categories**
   - Give SPF higher priority (health benefit)
   - Give serum higher priority (high margin)

### Medium Changes (Require Logic Updates)

1. **Add Bundle Detection**
   - Define bundle rules
   - Calculate bundle pricing
   - Score bundles vs. individual products

2. **Add Premium Upsells**
   - Map basic → premium products
   - Score premium vs. basic replacement

3. **Add Category-Based Filtering**
   - Don't recommend products customer explicitly doesn't want
   - Learn from customer behavior (never buys masks)

### Hard Changes (Require Architecture Changes)

1. **Add Machine Learning**
   - Collect conversion data
   - Train model on historical purchases
   - Replace or augment rule-based scoring

2. **Add Collaborative Filtering**
   - Build customer similarity matrix
   - Recommend based on similar customers

3. **Add Real-Time Personalization**
   - Track customer interactions
   - Adjust recommendations in real-time

---

## 💡 Summary: Why This Design?

**Strengths:**
- ✅ Fast and deterministic
- ✅ Transparent and explainable
- ✅ Easy to test and maintain
- ✅ Balances multiple business objectives
- ✅ Can evolve incrementally

**Trade-offs:**
- ❌ Less sophisticated than ML approaches
- ❌ Doesn't learn from customer behavior (yet)
- ❌ Fixed weights (could be adaptive)

**When to Evolve:**
- Add ML when you have conversion data
- Add LLMs for natural language rationale
- Add bundles when business model supports it
- Add personalization when you have interaction data

---

*This design follows orchestrator principles: rule-based first, transparent logic, incremental complexity.*



# business logic

In [ ]:
"""Business logic utilities for Cross-Sell & Upsell Orchestrator"""

from typing import Dict, List, Any, Optional
from datetime import datetime, timedelta
from .data_utils import get_essential_categories


def identify_routine_gaps(
    customer_categories: List[str],
    essential_categories: Optional[List[str]] = None
) -> List[str]:
    """
    Identify missing essential categories in customer's routine

    Args:
        customer_categories: List of categories customer currently has
        essential_categories: List of essential categories (defaults to Tier 1 essentials)

    Returns:
        List of missing essential categories
    """
    if essential_categories is None:
        essential_categories = get_essential_categories()

    gaps = [cat for cat in essential_categories if cat not in customer_categories]
    return gaps


def calculate_days_since_purchase(purchase_date: str) -> int:
    """
    Calculate days since purchase date

    Args:
        purchase_date: Date string in format "YYYY-MM-DD"

    Returns:
        Number of days since purchase
    """
    try:
        purchase_dt = datetime.strptime(purchase_date, "%Y-%m-%d")
        today = datetime.now()
        days_diff = (today - purchase_dt).days
        return max(0, days_diff)  # Don't return negative days
    except (ValueError, TypeError):
        return 0


def check_replenishment_needs(
    customer_products: List[Dict[str, Any]],
    product_catalog: List[Dict[str, Any]],
    product_lookup: Optional[Dict[str, Dict[str, Any]]] = None
) -> List[Dict[str, Any]]:
    """
    Check which customer products need replenishment

    Args:
        customer_products: List of customer's products with purchase_date
        product_catalog: Full product catalog
        product_lookup: Optional lookup dict for faster access

    Returns:
        List of products needing replenishment with details
    """
    if product_lookup is None:
        product_lookup = {p["product_id"]: p for p in product_catalog}

    replenishment_needs = []

    for product_owned in customer_products:
        product_id = product_owned.get("product_id")
        purchase_date = product_owned.get("purchase_date")

        if not product_id or not purchase_date:
            continue

        product_info = product_lookup.get(product_id)
        if not product_info:
            continue

        days_since_purchase = calculate_days_since_purchase(purchase_date)
        replenishment_cycle = product_info.get("replenishment_cycle_days", 30)

        days_until_replenishment = replenishment_cycle - days_since_purchase

        replenishment_needs.append({
            "product_id": product_id,
            "product_name": product_info.get("name", ""),
            "purchase_date": purchase_date,
            "days_since_purchase": days_since_purchase,
            "replenishment_cycle_days": replenishment_cycle,
            "days_until_replenishment": days_until_replenishment,
            "replenishment_due": days_until_replenishment <= 0,
            "approaching_replenishment": 0 < days_until_replenishment <= 7
        })

    return replenishment_needs


def find_cross_sell_opportunities(
    customer_data: Dict[str, Any],
    product_catalog: List[Dict[str, Any]],
    product_lookup: Dict[str, Dict[str, Any]],
    routine_gaps: List[str]
) -> List[Dict[str, Any]]:
    """
    Find cross-sell opportunities for customer

    Opportunities come from:
    1. Missing essential routine categories
    2. Recommended cross-sells from products customer owns

    Args:
        customer_data: Customer data dictionary
        product_catalog: Full product catalog
        product_lookup: product_id -> product dict lookup
        routine_gaps: List of missing essential categories

    Returns:
        List of cross-sell opportunities
    """
    opportunities = []
    customer_products = [p.get("product_id") for p in customer_data.get("products_owned", [])]
    customer_categories = customer_data.get("categories", [])

    # Track products we've already recommended to avoid duplicates
    recommended_product_ids = set()

    # 1. Fill routine gaps (essential categories)
    for gap_category in routine_gaps:
        # Find products in this category that customer doesn't own
        for product in product_catalog:
            product_id = product.get("product_id")
            product_category = product.get("category")

            if (product_category == gap_category and
                product_id not in customer_products and
                product_id not in recommended_product_ids):

                opportunities.append({
                    "product_id": product_id,
                    "product_name": product.get("name", ""),
                    "category": product_category,
                    "price": product.get("price", 0.0),
                    "margin": product.get("margin", "medium"),
                    "recommendation_type": "routine_gap",
                    "rationale": f"Customer missing essential {gap_category} step in routine"
                })
                recommended_product_ids.add(product_id)

    # 2. Product-based cross-sells (from recommended_cross_sells)
    for product_owned_id in customer_products:
        product_owned = product_lookup.get(product_owned_id)
        if not product_owned:
            continue

        recommended_cross_sells = product_owned.get("recommended_cross_sells", [])

        for recommended_id in recommended_cross_sells:
            # Skip if customer already owns it
            if recommended_id in customer_products:
                continue

            # Skip if we've already recommended it
            if recommended_id in recommended_product_ids:
                continue

            recommended_product = product_lookup.get(recommended_id)
            if not recommended_product:
                continue

            # Check if customer already has this category (optional products)
            recommended_category = recommended_product.get("category")
            if recommended_category in customer_categories:
                # Customer has this category, but this is a specific product recommendation
                # Still recommend it (could be an alternative or upgrade)
                pass

            opportunities.append({
                "product_id": recommended_id,
                "product_name": recommended_product.get("name", ""),
                "category": recommended_category,
                "price": recommended_product.get("price", 0.0),
                "margin": recommended_product.get("margin", "medium"),
                "recommendation_type": "product_cross_sell",
                "rationale": f"Recommended complement to {product_owned.get('name', product_owned_id)}"
            })
            recommended_product_ids.add(recommended_id)

    return opportunities


def find_upsell_opportunities(
    customer_data: Dict[str, Any],
    product_catalog: List[Dict[str, Any]],
    replenishment_needs: List[Dict[str, Any]]
) -> List[Dict[str, Any]]:
    """
    Find upsell opportunities for customer

    Upsell opportunities:
    1. Replenishment upsells (products needing refill)
    2. Bundle upsells (complete routine with missing products)

    Args:
        customer_data: Customer data dictionary
        product_catalog: Full product catalog
        replenishment_needs: Products needing replenishment

    Returns:
        List of upsell opportunities
    """
    opportunities = []
    customer_products = [p.get("product_id") for p in customer_data.get("products_owned", [])]

    # 1. Replenishment upsells
    for replenishment in replenishment_needs:
        product_id = replenishment.get("product_id")

        if product_id in customer_products:
            # Find product info
            product_info = next(
                (p for p in product_catalog if p.get("product_id") == product_id),
                None
            )

            if product_info:
                urgency_level = "high" if replenishment.get("replenishment_due") else "medium"

                opportunities.append({
                    "product_id": product_id,
                    "product_name": product_info.get("name", ""),
                    "category": product_info.get("category", ""),
                    "price": product_info.get("price", 0.0),
                    "margin": product_info.get("margin", "medium"),
                    "recommendation_type": "replenishment",
                    "rationale": f"Time to replenish {product_info.get('name', product_id)} - "
                                f"{replenishment.get('days_since_purchase')} days since purchase",
                    "replenishment_urgency": urgency_level,
                    "days_since_purchase": replenishment.get("days_since_purchase", 0)
                })

    return opportunities


def calculate_margin_multiplier(margin: str) -> float:
    """Convert margin string to multiplier for scoring"""
    margin_map = {
        "high": 1.5,
        "medium": 1.0,
        "low": 0.7
    }
    return margin_map.get(margin.lower(), 1.0)


def calculate_loyalty_multiplier(loyalty_tier: str) -> float:
    """Convert loyalty tier to multiplier for scoring"""
    tier_map = {
        "gold": 1.2,
        "silver": 1.0,
        "bronze": 0.8
    }
    return tier_map.get(loyalty_tier.lower(), 1.0)


def score_opportunity(
    opportunity: Dict[str, Any],
    customer_data: Dict[str, Any],
    product: Dict[str, Any],
    routine_gaps: List[str],
    replenishment_needs: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Score an opportunity based on business value, customer fit, routine completeness, and replenishment urgency

    Scoring weights:
    - Business Value: 40%
    - Customer Fit: 30%
    - Routine Completeness: 20%
    - Replenishment Urgency: 10%

    Args:
        opportunity: Opportunity dictionary
        customer_data: Customer data
        product: Product information
        routine_gaps: List of missing essential categories
        replenishment_needs: List of replenishment needs

    Returns:
        Opportunity dictionary with scoring fields added
    """
    # 1. Business Value Score (40%)
    price = opportunity.get("price", 0.0)
    margin = opportunity.get("margin", "medium")
    margin_multiplier = calculate_margin_multiplier(margin)
    business_value_score = price * margin_multiplier

    # 2. Customer Fit Score (30%)
    price_sensitivity = customer_data.get("price_sensitivity", "medium")
    loyalty_tier = customer_data.get("loyalty_tier", "bronze")
    churn_risk = customer_data.get("churn_risk", 0.5)

    # Price sensitivity match
    if price_sensitivity == "low":
        price_fit = 1.2  # Low sensitivity = higher prices OK
    elif price_sensitivity == "high":
        price_fit = 0.8  # High sensitivity = prefer lower prices
    else:
        price_fit = 1.0

    # Adjust price fit based on actual price (rough heuristic)
    if price > 18.0 and price_sensitivity == "high":
        price_fit *= 0.7  # Penalize high prices for price-sensitive customers
    elif price < 12.0 and price_sensitivity == "low":
        price_fit *= 0.9  # Slight penalty for low prices to low-sensitivity customers

    loyalty_multiplier = calculate_loyalty_multiplier(loyalty_tier)
    churn_urgency = 1.0 + (churn_risk * 0.3)  # Higher churn = more urgency

    customer_fit_score = (price_fit * loyalty_multiplier * churn_urgency) * 10  # Scale to ~10-20 range

    # 3. Routine Completeness Score (20%)
    category = opportunity.get("category", "")
    recommendation_type = opportunity.get("recommendation_type", "")

    if category in routine_gaps:
        # Essential category missing
        routine_score = 15.0  # High score for essential gaps
    elif recommendation_type == "routine_gap":
        routine_score = 12.0
    elif recommendation_type == "product_cross_sell":
        routine_score = 8.0
    else:
        routine_score = 5.0

    # 4. Replenishment Urgency Score (10%)
    if recommendation_type == "replenishment":
        days_since = opportunity.get("days_since_purchase", 0)
        if days_since > 30:  # Past due
            replenishment_score = 10.0
        elif days_since > 20:  # Approaching
            replenishment_score = 7.0
        else:
            replenishment_score = 5.0
    else:
        replenishment_score = 0.0

    # Calculate weighted final score
    final_score = (
        (business_value_score * 0.4) +
        (customer_fit_score * 0.3) +
        (routine_score * 0.2) +
        (replenishment_score * 0.1)
    )

    # Add scoring details to opportunity
    scored_opportunity = opportunity.copy()
    scored_opportunity.update({
        "raw_score": final_score,
        "business_value_score": business_value_score,
        "customer_fit_score": customer_fit_score,
        "routine_completeness_score": routine_score,
        "replenishment_urgency_score": replenishment_score
    })

    return scored_opportunity


def rank_opportunities(opportunities: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Rank opportunities by final score (descending)

    Args:
        opportunities: List of scored opportunities

    Returns:
        Sorted list of opportunities (highest score first)
    """
    return sorted(opportunities, key=lambda x: x.get("raw_score", 0.0), reverse=True)


def calculate_opportunity_summary(
    cross_sell_opportunities: List[Dict[str, Any]],
    upsell_opportunities: List[Dict[str, Any]],
    all_opportunities: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Calculate summary metrics for opportunities

    Args:
        cross_sell_opportunities: List of cross-sell opportunities
        upsell_opportunities: List of upsell opportunities
        all_opportunities: All scored and ranked opportunities

    Returns:
        Summary dictionary with metrics
    """
    total_potential_revenue = sum(opp.get("price", 0.0) for opp in all_opportunities)

    # Count high-value opportunities (score > 15)
    high_value_threshold = 15.0
    high_value_count = sum(1 for opp in all_opportunities if opp.get("raw_score", 0.0) > high_value_threshold)

    # Count replenishment urgency
    replenishment_urgent = sum(
        1 for opp in upsell_opportunities
        if opp.get("recommendation_type") == "replenishment" and opp.get("replenishment_urgency") == "high"
    )

    return {
        "total_cross_sell_opportunities": len(cross_sell_opportunities),
        "total_upsell_opportunities": len(upsell_opportunities),
        "total_opportunities": len(all_opportunities),
        "total_potential_revenue": round(total_potential_revenue, 2),
        "high_value_opportunities": high_value_count,
        "replenishment_urgency_count": replenishment_urgent
    }



# test business logic

In [ ]:
"""Tests for business logic utilities"""

import pytest
from pathlib import Path
import sys
from datetime import datetime, timedelta

# Add src to path
sys.path.insert(0, str(Path(__file__).parent.parent / "src"))

from cross_sell_upsell.business_logic import (
    identify_routine_gaps,
    calculate_days_since_purchase,
    check_replenishment_needs,
    find_cross_sell_opportunities,
    find_upsell_opportunities,
    score_opportunity,
    rank_opportunities,
    calculate_opportunity_summary
)
from cross_sell_upsell.data_utils import (
    load_customer_data,
    load_product_catalog,
    build_product_lookup
)


def test_identify_routine_gaps():
    """Test identifying missing essential categories"""
    # Customer with some categories
    customer_categories = ["cleanser", "moisturizer"]
    gaps = identify_routine_gaps(customer_categories)

    assert "toner" in gaps
    assert "serum" in gaps
    assert "spf" in gaps
    assert "cleanser" not in gaps
    assert "moisturizer" not in gaps


def test_identify_routine_gaps_complete():
    """Test customer with complete routine"""
    customer_categories = ["cleanser", "toner", "serum", "moisturizer", "spf"]
    gaps = identify_routine_gaps(customer_categories)
    assert len(gaps) == 0


def test_calculate_days_since_purchase():
    """Test calculating days since purchase"""
    # Purchase 30 days ago
    past_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
    days = calculate_days_since_purchase(past_date)
    assert days == 30

    # Today's date
    today = datetime.now().strftime("%Y-%m-%d")
    days = calculate_days_since_purchase(today)
    assert days == 0


def test_check_replenishment_needs():
    """Test checking replenishment needs"""
    catalog = load_product_catalog()
    product_lookup = build_product_lookup(catalog)

    # Customer product purchased 45 days ago (past 30-day cycle)
    customer_products = [
        {"product_id": "P001", "purchase_date": "2024-01-01", "amount": 14.99}
    ]

    needs = check_replenishment_needs(customer_products, catalog, product_lookup)

    assert len(needs) == 1
    assert needs[0]["product_id"] == "P001"
    assert needs[0]["replenishment_due"] == True  # Past 30 days
    assert needs[0]["days_since_purchase"] > 30


def test_find_cross_sell_opportunities():
    """Test finding cross-sell opportunities"""
    customer = load_customer_data("C001")  # Has cleanser, moisturizer
    catalog = load_product_catalog()
    product_lookup = build_product_lookup(catalog)
    routine_gaps = identify_routine_gaps(customer.get("categories", []))

    opportunities = find_cross_sell_opportunities(
        customer, catalog, product_lookup, routine_gaps
    )

    # Should find opportunities for missing categories (toner, serum, spf)
    # and product-based cross-sells
    assert len(opportunities) > 0

    # Check that we recommend products for missing categories
    opportunity_categories = [opp.get("category") for opp in opportunities]
    assert "toner" in opportunity_categories or "serum" in opportunity_categories or "spf" in opportunity_categories


def test_find_upsell_opportunities():
    """Test finding upsell opportunities"""
    customer = load_customer_data("C001")
    catalog = load_product_catalog()

    # Create replenishment needs
    customer_products = customer.get("products_owned", [])
    product_lookup = build_product_lookup(catalog)
    replenishment_needs = check_replenishment_needs(
        customer_products, catalog, product_lookup
    )

    opportunities = find_upsell_opportunities(
        customer, catalog, replenishment_needs
    )

    # Should find replenishment opportunities if products are due
    assert isinstance(opportunities, list)
    # May be empty if no products need replenishment yet


def test_score_opportunity():
    """Test scoring an opportunity"""
    customer = load_customer_data("C001")  # Gold tier, medium price sensitivity
    catalog = load_product_catalog()
    product = next(p for p in catalog if p["product_id"] == "P002")  # Toner

    opportunity = {
        "product_id": "P002",
        "product_name": "Balancing Facial Toner",
        "category": "toner",
        "price": 12.99,
        "margin": "medium",
        "recommendation_type": "routine_gap",
        "rationale": "Customer missing essential toner step"
    }

    routine_gaps = ["toner", "serum", "spf"]
    replenishment_needs = []

    scored = score_opportunity(
        opportunity, customer, product, routine_gaps, replenishment_needs
    )

    assert "raw_score" in scored
    assert "business_value_score" in scored
    assert "customer_fit_score" in scored
    assert "routine_completeness_score" in scored
    assert scored["raw_score"] > 0


def test_rank_opportunities():
    """Test ranking opportunities by score"""
    opportunities = [
        {"product_id": "P001", "raw_score": 10.0},
        {"product_id": "P002", "raw_score": 15.0},
        {"product_id": "P003", "raw_score": 8.0}
    ]

    ranked = rank_opportunities(opportunities)

    assert ranked[0]["product_id"] == "P002"  # Highest score first
    assert ranked[1]["product_id"] == "P001"
    assert ranked[2]["product_id"] == "P003"


def test_calculate_opportunity_summary():
    """Test calculating opportunity summary"""
    cross_sell = [
        {"product_id": "P001", "price": 14.99, "raw_score": 12.0},
        {"product_id": "P002", "price": 12.99, "raw_score": 18.0}
    ]
    upsell = [
        {"product_id": "P003", "price": 19.99, "raw_score": 10.0, "recommendation_type": "replenishment", "replenishment_urgency": "high"}
    ]
    all_opps = cross_sell + upsell

    summary = calculate_opportunity_summary(cross_sell, upsell, all_opps)

    assert summary["total_cross_sell_opportunities"] == 2
    assert summary["total_upsell_opportunities"] == 1
    assert summary["total_opportunities"] == 3
    assert summary["total_potential_revenue"] == pytest.approx(47.97, abs=0.01)
    assert summary["replenishment_urgency_count"] == 1



# Cross-Sell & Upsell Orchestrator Agent

In [ ]:
# ============================================================================
# Cross-Sell & Upsell Orchestrator Agent
# ============================================================================

class CrossSellUpsellState(TypedDict, total=False):
    """State for Cross-Sell & Upsell Orchestrator Agent"""

    # Input
    customer_id: str                        # Customer to analyze

    # Data Ingestion
    customer_data: Dict[str, Any]           # Loaded customer record
    product_catalog: List[Dict[str, Any]]   # All products
    product_lookup: Dict[str, Dict[str, Any]]  # product_id -> product dict (for fast lookup)

    # Routine Analysis
    customer_products: List[str]            # List of product_ids customer owns
    customer_categories: List[str]          # Categories customer has products in
    routine_gaps: List[str]                 # Missing essential categories
    replenishment_needs: List[Dict[str, Any]]  # Products needing replenishment
    # Structure: [{"product_id": "P001", "days_since_purchase": 45, "replenishment_due": True}]

    # Opportunity Detection
    cross_sell_opportunities: List[Dict[str, Any]]  # Cross-sell opportunities
    upsell_opportunities: List[Dict[str, Any]]      # Upsell opportunities
    # Structure per opportunity:
    # {
    #   "product_id": "P002",
    #   "product_name": "Balancing Facial Toner",
    #   "category": "toner",
    #   "price": 12.99,
    #   "margin": "medium",
    #   "recommendation_type": "routine_gap" | "product_cross_sell" | "replenishment",
    #   "rationale": "Customer missing essential toner step",
    #   "raw_score": float,
    #   "business_value_score": float,
    #   "customer_fit_score": float,
    #   "routine_completeness_score": float,
    #   "replenishment_urgency_score": float
    # }

    # Scoring & Ranking
    scored_opportunities: List[Dict[str, Any]]  # All opportunities with final scores
    ranked_opportunities: List[Dict[str, Any]]  # Sorted by final_score (descending)
    top_opportunities: List[Dict[str, Any]]     # Top N opportunities (configurable)

    # Summary Metrics
    opportunity_summary: Dict[str, Any]
    # Structure:
    # {
    #   "total_cross_sell_opportunities": int,
    #   "total_upsell_opportunities": int,
    #   "total_potential_revenue": float,
    #   "routine_completeness_percent": float,
    #   "replenishment_urgency_count": int,
    #   "high_value_opportunities": int  # Opportunities with score > threshold
    # }

    # Output
    recommendations_report: str             # Markdown report
    report_file_path: Optional[str]         # Path to saved report

    # Metadata
    errors: List[str]                       # Any errors encountered
    processing_time: Optional[float]        # Time taken to process


@dataclass
class CrossSellUpsellConfig:
    """Configuration for Cross-Sell & Upsell Orchestrator Agent"""
    llm_model: str = os.getenv("LLM_MODEL", "gpt-4o-mini")
    temperature: float = 0.3
    reports_dir: str = "output/cross_sell_reports"  # Where to save reports

    # Opportunity Ranking
    top_n_opportunities: int = 5  # Number of top opportunities to highlight

    # Scoring Thresholds (can be adjusted based on business needs)
    high_value_score_threshold: float = 15.0  # Opportunities with score > this are "high value"

    # Replenishment Settings
    replenishment_warning_days: int = 7  # Days before due date to flag as "approaching"



# State inspection and debugging utilities

In [ ]:
"""State inspection and debugging utilities for Cross-Sell & Upsell Orchestrator"""

import json
from typing import Dict, Any, Optional
from pathlib import Path


def print_state_summary(state: Dict[str, Any]) -> None:
    """
    Print human-readable state summary for debugging

    Args:
        state: CrossSellUpsellState dictionary
    """
    print("\n" + "="*60)
    print("CROSS-SELL & UPSELL ORCHESTRATOR - STATE SUMMARY")
    print("="*60)

    # Customer Info
    if "customer_id" in state:
        print(f"\n📋 Customer ID: {state['customer_id']}")

    if "customer_data" in state and state["customer_data"]:
        customer = state["customer_data"]
        print(f"   Name: {customer.get('name', 'N/A')}")
        print(f"   Loyalty Tier: {customer.get('loyalty_tier', 'N/A')}")
        print(f"   Churn Risk: {customer.get('churn_risk', 0.0):.2%}")
        print(f"   Lifetime Value: ${customer.get('lifetime_value', 0.0):.2f}")
        print(f"   Price Sensitivity: {customer.get('price_sensitivity', 'N/A')}")

    # Routine Analysis
    if "customer_categories" in state:
        print(f"\n🛍️  Current Products: {len(state.get('customer_products', []))} products")
        print(f"   Categories: {', '.join(state['customer_categories']) if state['customer_categories'] else 'None'}")

    if "routine_gaps" in state:
        gaps = state["routine_gaps"]
        print(f"\n⚠️  Routine Gaps: {len(gaps)} missing essential categories")
        if gaps:
            print(f"   Missing: {', '.join(gaps)}")
        else:
            print("   ✅ Complete routine!")

    if "replenishment_needs" in state:
        needs = state["replenishment_needs"]
        due = sum(1 for n in needs if n.get("replenishment_due", False))
        approaching = sum(1 for n in needs if n.get("approaching_replenishment", False))
        print(f"\n🔄 Replenishment Needs: {len(needs)} products")
        if due > 0:
            print(f"   ⚠️  {due} products past due")
        if approaching > 0:
            print(f"   ⏰ {approaching} products approaching due date")

    # Opportunities
    if "cross_sell_opportunities" in state:
        print(f"\n💡 Cross-Sell Opportunities: {len(state['cross_sell_opportunities'])}")

    if "upsell_opportunities" in state:
        print(f"   Upsell Opportunities: {len(state['upsell_opportunities'])}")

    if "ranked_opportunities" in state:
        ranked = state["ranked_opportunities"]
        print(f"\n⭐ Top Opportunities: {len(ranked)} total")
        if ranked:
            print(f"   #1: {ranked[0].get('product_name', 'N/A')} (Score: {ranked[0].get('raw_score', 0.0):.2f})")
            if len(ranked) > 1:
                print(f"   #2: {ranked[1].get('product_name', 'N/A')} (Score: {ranked[1].get('raw_score', 0.0):.2f})")
            if len(ranked) > 2:
                print(f"   #3: {ranked[2].get('product_name', 'N/A')} (Score: {ranked[2].get('raw_score', 0.0):.2f})")

    # Summary Metrics
    if "opportunity_summary" in state and state["opportunity_summary"]:
        summary = state["opportunity_summary"]
        print(f"\n📊 Summary Metrics:")
        print(f"   Total Opportunities: {summary.get('total_opportunities', 0)}")
        print(f"   Potential Revenue: ${summary.get('total_potential_revenue', 0.0):.2f}")
        print(f"   High-Value Opportunities: {summary.get('high_value_opportunities', 0)}")

    # Errors
    if "errors" in state and state["errors"]:
        print(f"\n❌ Errors: {len(state['errors'])}")
        for error in state["errors"]:
            print(f"   - {error}")

    # Processing Time
    if "processing_time" in state and state["processing_time"]:
        print(f"\n⏱️  Processing Time: {state['processing_time']:.2f}s")

    print("="*60 + "\n")


def save_state_to_json(state: Dict[str, Any], filename: str, output_dir: str = "output/debug") -> str:
    """
    Save state to JSON file for debugging

    Args:
        state: CrossSellUpsellState dictionary
        filename: Name of file to save (without .json extension)
        output_dir: Directory to save file in

    Returns:
        Path to saved file
    """
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)

    file_path = output_path / f"{filename}.json"

    # Convert state to JSON-serializable format
    # Handle any non-serializable types
    def json_serializer(obj):
        """Custom JSON serializer for objects not serializable by default json code"""
        if hasattr(obj, '__dict__'):
            return obj.__dict__
        raise TypeError(f"Type {type(obj)} not serializable")

    with open(file_path, 'w') as f:
        json.dump(state, f, indent=2, default=json_serializer)

    return str(file_path)


def print_opportunity_details(opportunity: Dict[str, Any], index: Optional[int] = None) -> None:
    """
    Print detailed information about a single opportunity

    Args:
        opportunity: Opportunity dictionary
        index: Optional index number (for ranking display)
    """
    prefix = f"#{index}: " if index is not None else ""
    print(f"\n{prefix}{opportunity.get('product_name', 'Unknown Product')}")
    print(f"  Product ID: {opportunity.get('product_id', 'N/A')}")
    print(f"  Category: {opportunity.get('category', 'N/A')}")
    print(f"  Price: ${opportunity.get('price', 0.0):.2f}")
    print(f"  Margin: {opportunity.get('margin', 'N/A')}")
    print(f"  Type: {opportunity.get('recommendation_type', 'N/A')}")
    print(f"  Rationale: {opportunity.get('rationale', 'N/A')}")

    if "raw_score" in opportunity:
        print(f"  📊 Scores:")
        print(f"     Final Score: {opportunity.get('raw_score', 0.0):.2f}")
        print(f"     Business Value: {opportunity.get('business_value_score', 0.0):.2f}")
        print(f"     Customer Fit: {opportunity.get('customer_fit_score', 0.0):.2f}")
        print(f"     Routine Completeness: {opportunity.get('routine_completeness_score', 0.0):.2f}")
        print(f"     Replenishment Urgency: {opportunity.get('replenishment_urgency_score', 0.0):.2f}")


def print_top_opportunities(state: Dict[str, Any], top_n: int = 5) -> None:
    """
    Print top N opportunities in detail

    Args:
        state: CrossSellUpsellState dictionary
        top_n: Number of top opportunities to display
    """
    if "ranked_opportunities" not in state or not state["ranked_opportunities"]:
        print("\n⚠️  No ranked opportunities available")
        return

    ranked = state["ranked_opportunities"]
    top_opportunities = ranked[:top_n]

    print(f"\n{'='*60}")
    print(f"TOP {len(top_opportunities)} OPPORTUNITIES")
    print(f"{'='*60}")

    for i, opp in enumerate(top_opportunities, 1):
        print_opportunity_details(opp, index=i)

    print(f"\n{'='*60}\n")



# Nodes

In [ ]:
"""LangGraph nodes for Cross-Sell & Upsell Orchestrator"""

from typing import Dict, Any
from .data_utils import (
    load_customer_data,
    load_product_catalog,
    build_product_lookup
)
from .business_logic import (
    identify_routine_gaps,
    check_replenishment_needs,
    find_cross_sell_opportunities,
    find_upsell_opportunities,
    score_opportunity,
    rank_opportunities,
    calculate_opportunity_summary
)
from .state_utils import print_state_summary


def data_ingestion_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Node 1: Load customer and product data

    Args:
        state: CrossSellUpsellState with customer_id

    Returns:
        Updated state with customer_data, product_catalog, product_lookup
    """
    customer_id = state.get("customer_id")
    if not customer_id:
        return {
            "errors": state.get("errors", []) + ["Missing customer_id in state"]
        }

    # Load customer data
    customer_data = load_customer_data(customer_id)
    if not customer_data:
        return {
            "errors": state.get("errors", []) + [f"Customer {customer_id} not found"]
        }

    # Load product catalog
    product_catalog = load_product_catalog()
    product_lookup = build_product_lookup(product_catalog)

    return {
        "customer_data": customer_data,
        "product_catalog": product_catalog,
        "product_lookup": product_lookup,
        "customer_products": [p.get("product_id") for p in customer_data.get("products_owned", [])],
        "customer_categories": customer_data.get("categories", [])
    }


def routine_analysis_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Node 2: Analyze customer routine and identify gaps

    Args:
        state: CrossSellUpsellState with customer_data, product_catalog, etc.

    Returns:
        Updated state with routine_gaps and replenishment_needs
    """
    customer_data = state.get("customer_data")
    customer_products = state.get("customer_products", [])
    product_catalog = state.get("product_catalog", [])
    product_lookup = state.get("product_lookup", {})
    customer_categories = state.get("customer_categories", [])

    if not customer_data:
        return {
            "errors": state.get("errors", []) + ["Missing customer_data in state"]
        }

    # Identify routine gaps
    routine_gaps = identify_routine_gaps(customer_categories)

    # Check replenishment needs
    # Need to get full product dicts with purchase dates
    customer_products_with_dates = customer_data.get("products_owned", [])
    replenishment_needs = check_replenishment_needs(
        customer_products_with_dates,
        product_catalog,
        product_lookup
    )

    return {
        "routine_gaps": routine_gaps,
        "replenishment_needs": replenishment_needs
    }


def opportunity_detection_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Node 3: Find cross-sell and upsell opportunities

    Args:
        state: CrossSellUpsellState with routine analysis complete

    Returns:
        Updated state with cross_sell_opportunities and upsell_opportunities
    """
    customer_data = state.get("customer_data")
    product_catalog = state.get("product_catalog", [])
    product_lookup = state.get("product_lookup", {})
    routine_gaps = state.get("routine_gaps", [])
    replenishment_needs = state.get("replenishment_needs", [])

    if not customer_data:
        return {
            "errors": state.get("errors", []) + ["Missing customer_data in state"]
        }

    # Find cross-sell opportunities
    cross_sell_opportunities = find_cross_sell_opportunities(
        customer_data,
        product_catalog,
        product_lookup,
        routine_gaps
    )

    # Find upsell opportunities
    upsell_opportunities = find_upsell_opportunities(
        customer_data,
        product_catalog,
        replenishment_needs
    )

    return {
        "cross_sell_opportunities": cross_sell_opportunities,
        "upsell_opportunities": upsell_opportunities
    }


def scoring_ranking_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Node 4: Score and rank all opportunities

    Args:
        state: CrossSellUpsellState with opportunities detected

    Returns:
        Updated state with scored_opportunities, ranked_opportunities, and summary
    """
    customer_data = state.get("customer_data")
    product_lookup = state.get("product_lookup", {})
    routine_gaps = state.get("routine_gaps", [])
    replenishment_needs = state.get("replenishment_needs", [])
    cross_sell_opportunities = state.get("cross_sell_opportunities", [])
    upsell_opportunities = state.get("upsell_opportunities", [])

    if not customer_data:
        return {
            "errors": state.get("errors", []) + ["Missing customer_data in state"]
        }

    # Score all opportunities
    all_opportunities = cross_sell_opportunities + upsell_opportunities
    scored_opportunities = []

    for opportunity in all_opportunities:
        product_id = opportunity.get("product_id")
        product = product_lookup.get(product_id, {})

        scored_opp = score_opportunity(
            opportunity,
            customer_data,
            product,
            routine_gaps,
            replenishment_needs
        )
        scored_opportunities.append(scored_opp)

    # Rank opportunities
    ranked_opportunities = rank_opportunities(scored_opportunities)

    # Calculate summary
    opportunity_summary = calculate_opportunity_summary(
        cross_sell_opportunities,
        upsell_opportunities,
        ranked_opportunities
    )

    # Calculate routine completeness percentage
    essential_categories = ["cleanser", "toner", "serum", "moisturizer", "spf"]
    customer_categories = state.get("customer_categories", [])
    categories_owned = sum(1 for cat in essential_categories if cat in customer_categories)
    routine_completeness_percent = (categories_owned / len(essential_categories)) * 100
    opportunity_summary["routine_completeness_percent"] = round(routine_completeness_percent, 1)

    return {
        "scored_opportunities": scored_opportunities,
        "ranked_opportunities": ranked_opportunities,
        "opportunity_summary": opportunity_summary
    }


def report_generation_node(state: Dict[str, Any]) -> Dict[str, Any]:
    """
    Node 5: Generate markdown report

    Args:
        state: CrossSellUpsellState with all analysis complete

    Returns:
        Updated state with recommendations_report
    """
    customer_data = state.get("customer_data", {})
    ranked_opportunities = state.get("ranked_opportunities", [])
    opportunity_summary = state.get("opportunity_summary", {})
    routine_gaps = state.get("routine_gaps", [])
    replenishment_needs = state.get("replenishment_needs", [])

    # Build report
    report_lines = []

    # Header
    report_lines.append("# Cross-Sell & Upsell Recommendations Report\n")
    report_lines.append(f"**Customer:** {customer_data.get('name', 'N/A')} ({customer_data.get('customer_id', 'N/A')})\n")
    report_lines.append(f"**Generated:** {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    # Customer Overview
    report_lines.append("## Customer Overview\n")
    report_lines.append(f"- **Loyalty Tier:** {customer_data.get('loyalty_tier', 'N/A').title()}")
    report_lines.append(f"- **Lifetime Value:** ${customer_data.get('lifetime_value', 0.0):.2f}")
    report_lines.append(f"- **Churn Risk:** {customer_data.get('churn_risk', 0.0):.1%}")
    report_lines.append(f"- **Price Sensitivity:** {customer_data.get('price_sensitivity', 'N/A').title()}")
    report_lines.append(f"- **Current Products:** {len(state.get('customer_products', []))} products")
    report_lines.append(f"- **Routine Completeness:** {opportunity_summary.get('routine_completeness_percent', 0.0):.1f}%\n")

    # Routine Analysis
    report_lines.append("## Routine Analysis\n")
    if routine_gaps:
        report_lines.append(f"**Missing Essential Categories:** {', '.join(routine_gaps)}\n")
    else:
        report_lines.append("✅ **Complete Routine** - Customer has all essential products!\n")

    if replenishment_needs:
        due_count = sum(1 for n in replenishment_needs if n.get("replenishment_due", False))
        if due_count > 0:
            report_lines.append(f"⚠️  **{due_count} products past replenishment date**\n")

    # Opportunities Summary
    report_lines.append("## Opportunities Summary\n")
    report_lines.append(f"- **Total Cross-Sell Opportunities:** {opportunity_summary.get('total_cross_sell_opportunities', 0)}")
    report_lines.append(f"- **Total Upsell Opportunities:** {opportunity_summary.get('total_upsell_opportunities', 0)}")
    report_lines.append(f"- **Total Potential Revenue:** ${opportunity_summary.get('total_potential_revenue', 0.0):.2f}")
    report_lines.append(f"- **High-Value Opportunities:** {opportunity_summary.get('high_value_opportunities', 0)}\n")

    # Top Recommendations
    report_lines.append("## Top Recommendations\n")
    if ranked_opportunities:
        top_n = min(5, len(ranked_opportunities))
        for i, opp in enumerate(ranked_opportunities[:top_n], 1):
            report_lines.append(f"### {i}. {opp.get('product_name', 'Unknown Product')}")
            report_lines.append(f"**Category:** {opp.get('category', 'N/A').title()}")
            report_lines.append(f"**Price:** ${opp.get('price', 0.0):.2f}")
            report_lines.append(f"**Type:** {opp.get('recommendation_type', 'N/A').replace('_', ' ').title()}")
            report_lines.append(f"**Rationale:** {opp.get('rationale', 'N/A')}")

            if "raw_score" in opp:
                report_lines.append(f"**Score:** {opp.get('raw_score', 0.0):.2f}")
                report_lines.append(f"  - Business Value: {opp.get('business_value_score', 0.0):.2f}")
                report_lines.append(f"  - Customer Fit: {opp.get('customer_fit_score', 0.0):.2f}")
                report_lines.append(f"  - Routine Completeness: {opp.get('routine_completeness_score', 0.0):.2f}")
                report_lines.append(f"  - Replenishment Urgency: {opp.get('replenishment_urgency_score', 0.0):.2f}")

            report_lines.append("")  # Blank line
    else:
        report_lines.append("No opportunities found.\n")

    # All Opportunities (if more than top 5)
    if len(ranked_opportunities) > 5:
        report_lines.append("## All Opportunities\n")
        report_lines.append(f"*Showing top 5 above. Total of {len(ranked_opportunities)} opportunities found.*\n")

    report = "\n".join(report_lines)

    return {
        "recommendations_report": report
    }



# Workflow

In [ ]:
"""LangGraph workflow for Cross-Sell & Upsell Orchestrator"""

from langgraph.graph import StateGraph, END
from typing import Dict, Any
import sys
from pathlib import Path

# Add parent directory to path to import config
sys.path.insert(0, str(Path(__file__).parent.parent.parent))

from config import CrossSellUpsellState
from .nodes import (
    data_ingestion_node,
    routine_analysis_node,
    opportunity_detection_node,
    scoring_ranking_node,
    report_generation_node
)


def create_cross_sell_upsell_workflow() -> StateGraph:
    """
    Create and configure the Cross-Sell & Upsell Orchestrator workflow

    Returns:
        Configured StateGraph ready to compile
    """
    # Create workflow graph
    workflow = StateGraph(CrossSellUpsellState)

    # Add nodes
    workflow.add_node("data_ingestion", data_ingestion_node)
    workflow.add_node("routine_analysis", routine_analysis_node)
    workflow.add_node("opportunity_detection", opportunity_detection_node)
    workflow.add_node("scoring_ranking", scoring_ranking_node)
    workflow.add_node("report_generation", report_generation_node)

    # Define linear flow
    workflow.set_entry_point("data_ingestion")
    workflow.add_edge("data_ingestion", "routine_analysis")
    workflow.add_edge("routine_analysis", "opportunity_detection")
    workflow.add_edge("opportunity_detection", "scoring_ranking")
    workflow.add_edge("scoring_ranking", "report_generation")
    workflow.add_edge("report_generation", END)

    return workflow


def run_workflow(customer_id: str, verbose: bool = True) -> Dict[str, Any]:
    """
    Run the Cross-Sell & Upsell Orchestrator workflow for a customer

    Args:
        customer_id: Customer ID to analyze (e.g., "C001")
        verbose: Whether to print state summaries during execution

    Returns:
        Final state dictionary
    """
    from .state_utils import print_state_summary
    import time

    # Create and compile workflow
    workflow = create_cross_sell_upsell_workflow()
    app = workflow.compile()

    # Initialize state
    initial_state: CrossSellUpsellState = {
        "customer_id": customer_id,
        "errors": []
    }

    # Run workflow
    start_time = time.time()

    if verbose:
        print(f"\n🚀 Starting Cross-Sell & Upsell Orchestrator for customer {customer_id}")
        print("="*60)

    try:
        # Execute workflow
        final_state = app.invoke(initial_state)

        # Calculate processing time
        processing_time = time.time() - start_time
        final_state["processing_time"] = processing_time

        if verbose:
            print_state_summary(final_state)

        return final_state

    except Exception as e:
        error_state = {
            **initial_state,
            "errors": [f"Workflow execution failed: {str(e)}"],
            "processing_time": time.time() - start_time
        }
        if verbose:
            print(f"\n❌ Error: {str(e)}")
        return error_state



In [ ]:
"""Simple script to run the Cross-Sell & Upsell Orchestrator"""

import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path(__file__).parent / "src"))

from cross_sell_upsell.workflow import run_workflow
from cross_sell_upsell.state_utils import print_top_opportunities, save_state_to_json


def main():
    """Run orchestrator for a customer"""

    # Default customer ID (can be changed)
    customer_id = "C001"

    # Allow command line argument
    if len(sys.argv) > 1:
        customer_id = sys.argv[1]

    print(f"\n{'='*60}")
    print(f"CROSS-SELL & UPSELL ORCHESTRATOR")
    print(f"{'='*60}")
    print(f"Analyzing customer: {customer_id}\n")

    # Run workflow
    final_state = run_workflow(customer_id, verbose=True)

    # Print top opportunities
    if "ranked_opportunities" in final_state and final_state["ranked_opportunities"]:
        print_top_opportunities(final_state, top_n=5)

    # Save state for debugging (optional)
    if "--save-state" in sys.argv:
        output_file = save_state_to_json(final_state, f"state_{customer_id}")
        print(f"💾 State saved to: {output_file}\n")

    # Print report preview
    if "recommendations_report" in final_state:
        print("\n" + "="*60)
        print("REPORT PREVIEW (first 500 chars)")
        print("="*60)
        report = final_state["recommendations_report"]
        print(report[:500] + "..." if len(report) > 500 else report)
        print("\n" + "="*60)

    # Check for errors
    if final_state.get("errors"):
        print(f"\n⚠️  Errors encountered: {len(final_state['errors'])}")
        for error in final_state["errors"]:
            print(f"   - {error}")

    print(f"\n✅ Orchestrator completed in {final_state.get('processing_time', 0):.2f}s\n")


if __name__ == "__main__":
    main()



# Analyzing customer: C001

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_029_CrossSell_Upsell_Orchestrator % python run_cross_sell_orchestrator.py C001

============================================================
CROSS-SELL & UPSELL ORCHESTRATOR
============================================================
Analyzing customer: C001


🚀 Starting Cross-Sell & Upsell Orchestrator for customer C001
============================================================
/Users/micahshull/Documents/AI_LangGraph/LG_Cursor_029_CrossSell_Upsell_Orchestrator/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)

============================================================
CROSS-SELL & UPSELL ORCHESTRATOR - STATE SUMMARY
============================================================

📋 Customer ID: C001
   Name: Sarah Lee
   Loyalty Tier: gold
   Churn Risk: 12.00%
   Lifetime Value: $210.40
   Price Sensitivity: medium

🛍️  Current Products: 2 products
   Categories: cleanser, moisturizer

⚠️  Routine Gaps: 3 missing essential categories
   Missing: toner, serum, spf

🔄 Replenishment Needs: 2 products
   ⚠️  2 products past due

💡 Cross-Sell Opportunities: 3
   Upsell Opportunities: 2

⭐ Top Opportunities: 5 total
   #1: Hydrating Hyaluronic Serum (Score: 18.72)
   #2: SPF 30 Everyday Sunscreen (Score: 13.13)
   #3: Daily Lightweight Moisturizer (Score: 12.93)

📊 Summary Metrics:
   Total Opportunities: 5
   Potential Revenue: $81.95
   High-Value Opportunities: 1

⏱️  Processing Time: 0.05s
============================================================


============================================================
TOP 5 OPPORTUNITIES
============================================================

#1: Hydrating Hyaluronic Serum
  Product ID: P003
  Category: serum
  Price: $19.99
  Margin: high
  Type: routine_gap
  Rationale: Customer missing essential serum step in routine
  📊 Scores:
     Final Score: 18.72
     Business Value: 29.98
     Customer Fit: 12.43
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#2: SPF 30 Everyday Sunscreen
  Product ID: P005
  Category: spf
  Price: $15.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential spf step in routine
  📊 Scores:
     Final Score: 13.13
     Business Value: 15.99
     Customer Fit: 12.43
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#3: Daily Lightweight Moisturizer
  Product ID: P004
  Category: moisturizer
  Price: $17.99
  Margin: medium
  Type: replenishment
  Rationale: Time to replenish Daily Lightweight Moisturizer - 656 days since purchase
  📊 Scores:
     Final Score: 12.93
     Business Value: 17.99
     Customer Fit: 12.43
     Routine Completeness: 5.00
     Replenishment Urgency: 10.00

#4: Balancing Facial Toner
  Product ID: P002
  Category: toner
  Price: $12.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential toner step in routine
  📊 Scores:
     Final Score: 11.93
     Business Value: 12.99
     Customer Fit: 12.43
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#5: Gentle Foaming Cleanser
  Product ID: P001
  Category: cleanser
  Price: $14.99
  Margin: medium
  Type: replenishment
  Rationale: Time to replenish Gentle Foaming Cleanser - 680 days since purchase
  📊 Scores:
     Final Score: 11.73
     Business Value: 14.99
     Customer Fit: 12.43
     Routine Completeness: 5.00
     Replenishment Urgency: 10.00

============================================================


============================================================
REPORT PREVIEW (first 500 chars)
============================================================
# Cross-Sell & Upsell Recommendations Report

**Customer:** Sarah Lee (C001)

**Generated:** 2025-11-20 16:26:37

## Customer Overview

- **Loyalty Tier:** Gold
- **Lifetime Value:** $210.40
- **Churn Risk:** 12.0%
- **Price Sensitivity:** Medium
- **Current Products:** 2 products
- **Routine Completeness:** 40.0%

## Routine Analysis

**Missing Essential Categories:** toner, serum, spf

⚠️  **2 products past replenishment date**

## Opportunities Summary

- **Total Cross-Sell Opportunities:** 3...

============================================================

✅ Orchestrator completed in 0.05s

(.venv) micahshull@Micahs-iMac LG_Cursor_029_CrossSell_Upsell_Orchestrator %

# Analyzing customer: C002

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_029_CrossSell_Upsell_Orchestrator % python run_cross_sell_orchestrator.py C002

============================================================
CROSS-SELL & UPSELL ORCHESTRATOR
============================================================
Analyzing customer: C002


🚀 Starting Cross-Sell & Upsell Orchestrator for customer C002
============================================================
/Users/micahshull/Documents/AI_LangGraph/LG_Cursor_029_CrossSell_Upsell_Orchestrator/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)

============================================================
CROSS-SELL & UPSELL ORCHESTRATOR - STATE SUMMARY
============================================================

📋 Customer ID: C002
   Name: Mark Johnson
   Loyalty Tier: silver
   Churn Risk: 28.00%
   Lifetime Value: $89.50
   Price Sensitivity: high

🛍️  Current Products: 1 products
   Categories: toner

⚠️  Routine Gaps: 4 missing essential categories
   Missing: cleanser, serum, moisturizer, spf

🔄 Replenishment Needs: 1 products
   ⚠️  1 products past due

💡 Cross-Sell Opportunities: 5
   Upsell Opportunities: 1

⭐ Top Opportunities: 6 total
   #1: Hydrating Hyaluronic Serum (Score: 16.82)
   #2: Daily Lightweight Moisturizer (Score: 12.80)
   #3: SPF 30 Everyday Sunscreen (Score: 12.00)

📊 Summary Metrics:
   Total Opportunities: 6
   Potential Revenue: $95.94
   High-Value Opportunities: 1

⏱️  Processing Time: 0.04s
============================================================


============================================================
TOP 5 OPPORTUNITIES
============================================================

#1: Hydrating Hyaluronic Serum
  Product ID: P003
  Category: serum
  Price: $19.99
  Margin: high
  Type: routine_gap
  Rationale: Customer missing essential serum step in routine
  📊 Scores:
     Final Score: 16.82
     Business Value: 29.98
     Customer Fit: 6.07
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#2: Daily Lightweight Moisturizer
  Product ID: P004
  Category: moisturizer
  Price: $17.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential moisturizer step in routine
  📊 Scores:
     Final Score: 12.80
     Business Value: 17.99
     Customer Fit: 8.67
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#3: SPF 30 Everyday Sunscreen
  Product ID: P005
  Category: spf
  Price: $15.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential spf step in routine
  📊 Scores:
     Final Score: 12.00
     Business Value: 15.99
     Customer Fit: 8.67
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#4: Gentle Foaming Cleanser
  Product ID: P001
  Category: cleanser
  Price: $14.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential cleanser step in routine
  📊 Scores:
     Final Score: 11.60
     Business Value: 14.99
     Customer Fit: 8.67
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#5: Calming Chamomile Cleanser
  Product ID: P010
  Category: cleanser
  Price: $13.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential cleanser step in routine
  📊 Scores:
     Final Score: 11.20
     Business Value: 13.99
     Customer Fit: 8.67
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

============================================================


============================================================
REPORT PREVIEW (first 500 chars)
============================================================
# Cross-Sell & Upsell Recommendations Report

**Customer:** Mark Johnson (C002)

**Generated:** 2025-11-20 16:27:36

## Customer Overview

- **Loyalty Tier:** Silver
- **Lifetime Value:** $89.50
- **Churn Risk:** 28.0%
- **Price Sensitivity:** High
- **Current Products:** 1 products
- **Routine Completeness:** 20.0%

## Routine Analysis

**Missing Essential Categories:** cleanser, serum, moisturizer, spf

⚠️  **1 products past replenishment date**

## Opportunities Summary

- **Total Cross-Sell ...

============================================================

✅ Orchestrator completed in 0.04s


# Analyzing customer: C003

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_029_CrossSell_Upsell_Orchestrator % python run_cross_sell_orchestrator.py C002
python run_cross_sell_orchestrator.py C003

============================================================
CROSS-SELL & UPSELL ORCHESTRATOR
============================================================
Analyzing customer: C002


🚀 Starting Cross-Sell & Upsell Orchestrator for customer C002
============================================================
/Users/micahshull/Documents/AI_LangGraph/LG_Cursor_029_CrossSell_Upsell_Orchestrator/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)

============================================================
CROSS-SELL & UPSELL ORCHESTRATOR - STATE SUMMARY
============================================================

📋 Customer ID: C002
   Name: Mark Johnson
   Loyalty Tier: silver
   Churn Risk: 28.00%
   Lifetime Value: $89.50
   Price Sensitivity: high

🛍️  Current Products: 1 products
   Categories: toner

⚠️  Routine Gaps: 4 missing essential categories
   Missing: cleanser, serum, moisturizer, spf

🔄 Replenishment Needs: 1 products
   ⚠️  1 products past due

💡 Cross-Sell Opportunities: 5
   Upsell Opportunities: 1

⭐ Top Opportunities: 6 total
   #1: Hydrating Hyaluronic Serum (Score: 16.82)
   #2: Daily Lightweight Moisturizer (Score: 12.80)
   #3: SPF 30 Everyday Sunscreen (Score: 12.00)

📊 Summary Metrics:
   Total Opportunities: 6
   Potential Revenue: $95.94
   High-Value Opportunities: 1

⏱️  Processing Time: 0.03s
============================================================


============================================================
TOP 5 OPPORTUNITIES
============================================================

#1: Hydrating Hyaluronic Serum
  Product ID: P003
  Category: serum
  Price: $19.99
  Margin: high
  Type: routine_gap
  Rationale: Customer missing essential serum step in routine
  📊 Scores:
     Final Score: 16.82
     Business Value: 29.98
     Customer Fit: 6.07
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#2: Daily Lightweight Moisturizer
  Product ID: P004
  Category: moisturizer
  Price: $17.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential moisturizer step in routine
  📊 Scores:
     Final Score: 12.80
     Business Value: 17.99
     Customer Fit: 8.67
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#3: SPF 30 Everyday Sunscreen
  Product ID: P005
  Category: spf
  Price: $15.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential spf step in routine
  📊 Scores:
     Final Score: 12.00
     Business Value: 15.99
     Customer Fit: 8.67
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#4: Gentle Foaming Cleanser
  Product ID: P001
  Category: cleanser
  Price: $14.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential cleanser step in routine
  📊 Scores:
     Final Score: 11.60
     Business Value: 14.99
     Customer Fit: 8.67
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#5: Calming Chamomile Cleanser
  Product ID: P010
  Category: cleanser
  Price: $13.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential cleanser step in routine
  📊 Scores:
     Final Score: 11.20
     Business Value: 13.99
     Customer Fit: 8.67
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

============================================================


============================================================
REPORT PREVIEW (first 500 chars)
============================================================
# Cross-Sell & Upsell Recommendations Report

**Customer:** Mark Johnson (C002)

**Generated:** 2025-11-20 16:29:23

## Customer Overview

- **Loyalty Tier:** Silver
- **Lifetime Value:** $89.50
- **Churn Risk:** 28.0%
- **Price Sensitivity:** High
- **Current Products:** 1 products
- **Routine Completeness:** 20.0%

## Routine Analysis

**Missing Essential Categories:** cleanser, serum, moisturizer, spf

⚠️  **1 products past replenishment date**

## Opportunities Summary

- **Total Cross-Sell ...

============================================================

✅ Orchestrator completed in 0.03s


============================================================
CROSS-SELL & UPSELL ORCHESTRATOR
============================================================
Analyzing customer: C003


🚀 Starting Cross-Sell & Upsell Orchestrator for customer C003
============================================================
/Users/micahshull/Documents/AI_LangGraph/LG_Cursor_029_CrossSell_Upsell_Orchestrator/.venv/lib/python3.13/site-packages/pydantic/v1/main.py:1054: UserWarning: LangSmith now uses UUID v7 for run and trace identifiers. This warning appears when passing custom IDs. Please use: from langsmith import uuid7
            id = uuid7()
Future versions will require UUID v7.
  input_data = validator(cls_, input_data)

============================================================
CROSS-SELL & UPSELL ORCHESTRATOR - STATE SUMMARY
============================================================

📋 Customer ID: C003
   Name: Emily Chen
   Loyalty Tier: bronze
   Churn Risk: 8.00%
   Lifetime Value: $142.10
   Price Sensitivity: medium

🛍️  Current Products: 2 products
   Categories: serum, lip

⚠️  Routine Gaps: 4 missing essential categories
   Missing: cleanser, toner, moisturizer, spf

🔄 Replenishment Needs: 2 products
   ⚠️  2 products past due

💡 Cross-Sell Opportunities: 5
   Upsell Opportunities: 2

⭐ Top Opportunities: 7 total
   #1: Hydrating Hyaluronic Serum (Score: 16.45)
   #2: Daily Lightweight Moisturizer (Score: 12.65)
   #3: SPF 30 Everyday Sunscreen (Score: 11.85)

📊 Summary Metrics:
   Total Opportunities: 7
   Potential Revenue: $102.93
   High-Value Opportunities: 1

⏱️  Processing Time: 0.02s
============================================================


============================================================
TOP 5 OPPORTUNITIES
============================================================

#1: Hydrating Hyaluronic Serum
  Product ID: P003
  Category: serum
  Price: $19.99
  Margin: high
  Type: replenishment
  Rationale: Time to replenish Hydrating Hyaluronic Serum - 648 days since purchase
  📊 Scores:
     Final Score: 16.45
     Business Value: 29.98
     Customer Fit: 8.19
     Routine Completeness: 5.00
     Replenishment Urgency: 10.00

#2: Daily Lightweight Moisturizer
  Product ID: P004
  Category: moisturizer
  Price: $17.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential moisturizer step in routine
  📊 Scores:
     Final Score: 12.65
     Business Value: 17.99
     Customer Fit: 8.19
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#3: SPF 30 Everyday Sunscreen
  Product ID: P005
  Category: spf
  Price: $15.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential spf step in routine
  📊 Scores:
     Final Score: 11.85
     Business Value: 15.99
     Customer Fit: 8.19
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#4: Gentle Foaming Cleanser
  Product ID: P001
  Category: cleanser
  Price: $14.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential cleanser step in routine
  📊 Scores:
     Final Score: 11.45
     Business Value: 14.99
     Customer Fit: 8.19
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

#5: Calming Chamomile Cleanser
  Product ID: P010
  Category: cleanser
  Price: $13.99
  Margin: medium
  Type: routine_gap
  Rationale: Customer missing essential cleanser step in routine
  📊 Scores:
     Final Score: 11.05
     Business Value: 13.99
     Customer Fit: 8.19
     Routine Completeness: 15.00
     Replenishment Urgency: 0.00

============================================================


============================================================
REPORT PREVIEW (first 500 chars)
============================================================
# Cross-Sell & Upsell Recommendations Report

**Customer:** Emily Chen (C003)

**Generated:** 2025-11-20 16:29:24

## Customer Overview

- **Loyalty Tier:** Bronze
- **Lifetime Value:** $142.10
- **Churn Risk:** 8.0%
- **Price Sensitivity:** Medium
- **Current Products:** 2 products
- **Routine Completeness:** 20.0%

## Routine Analysis

**Missing Essential Categories:** cleanser, toner, moisturizer, spf

⚠️  **2 products past replenishment date**

## Opportunities Summary

- **Total Cross-Sell ...

============================================================

✅ Orchestrator completed in 0.02s

(.venv) micahshull@Micahs-iMac LG_Cursor_029_CrossSell_Upsell_Orchestrator %

# Cross-Sell & Upsell Orchestrator - Complete! 🎉

**Status:** MVP Complete - Ready for Testing

---

## ✅ What We've Built

### **Foundation (Architecture)**
1. ✅ **Data Utilities** (`src/cross_sell_upsell/data_utils.py`)
   - Load customer data
   - Load product catalog
   - Build fast lookup dictionaries

2. ✅ **Business Logic Utilities** (`src/cross_sell_upsell/business_logic.py`)
   - Routine gap detection
   - Replenishment checking
   - Cross-sell opportunity finding
   - Upsell opportunity finding
   - Opportunity scoring (multi-dimensional)
   - Ranking and summary calculation

3. ✅ **State Schema** (`config.py`)
   - `CrossSellUpsellState` TypedDict
   - `CrossSellUpsellConfig` dataclass

4. ✅ **State Inspection Tools** (`src/cross_sell_upsell/state_utils.py`)
   - Print state summaries
   - Save state to JSON
   - Print opportunity details

### **Orchestration (Nodes)**
5. ✅ **Data Ingestion Node** - Loads customer and product data
6. ✅ **Routine Analysis Node** - Identifies gaps and replenishment needs
7. ✅ **Opportunity Detection Node** - Finds cross-sell and upsell opportunities
8. ✅ **Scoring & Ranking Node** - Scores and ranks all opportunities
9. ✅ **Report Generation Node** - Generates markdown report

### **Workflow**
10. ✅ **LangGraph Workflow** (`src/cross_sell_upsell/workflow.py`)
    - Linear flow: Data → Analysis → Detection → Scoring → Report
    - Ready to run!

### **Testing & Execution**
11. ✅ **Test Files** - Unit tests for utilities
12. ✅ **Run Script** (`run_cross_sell_orchestrator.py`) - Easy execution

---

## 🏗️ Architecture Pattern

```
┌─────────────────────────────────────────────────────────┐
│                    LangGraph Workflow                     │
│  (Linear: Data → Analysis → Detection → Scoring → Report) │
└─────────────────────────────────────────────────────────┘
                          │
        ┌─────────────────┼─────────────────┐
        │                 │                 │
        ▼                 ▼                 ▼
┌──────────────┐  ┌──────────────┐  ┌──────────────┐
│    Nodes     │  │  Utilities   │  │ State Tools   │
│ (Orchestrate)│  │ (Business    │  │ (Inspection)  │
│              │  │   Logic)     │  │               │
└──────────────┘  └──────────────┘  └──────────────┘
```

**Key Principle:** Nodes orchestrate, utilities do the work, state tools help debug.

---

## 🚀 How to Run

### **Quick Test**
```bash
python run_cross_sell_orchestrator.py C001
```

### **Test Different Customers**
```bash
python run_cross_sell_orchestrator.py C002
python run_cross_sell_orchestrator.py C003
```

### **Save State for Debugging**
```bash
python run_cross_sell_orchestrator.py C001 --save-state
```

### **Run Tests**
```bash
pytest tests/test_data_utils.py -v
pytest tests/test_business_logic.py -v
```

---

## 📊 What the Orchestrator Does

1. **Loads Data** - Customer profile + product catalog
2. **Analyzes Routine** - Identifies missing essential categories
3. **Checks Replenishment** - Finds products needing refill
4. **Finds Opportunities** - Cross-sell (new products) + Upsell (replenishment)
5. **Scores Everything** - Multi-dimensional scoring (business value, customer fit, routine completeness, urgency)
6. **Ranks Opportunities** - Sorts by final score
7. **Generates Report** - Markdown report with recommendations

---

## 🎯 Example Output

For customer C001 (Sarah Lee - has cleanser + moisturizer):

**Routine Gaps:** toner, serum, spf
**Replenishment:** None (recent purchases)
**Top Recommendations:**
1. Balancing Facial Toner (routine gap, high score)
2. Hydrating Hyaluronic Serum (routine gap, high margin)
3. SPF 30 Everyday Sunscreen (routine gap, essential)

**Potential Revenue:** $50-60 from completing routine

---

## 🔧 What Can Be Enhanced

### **Easy Enhancements** (No Architecture Changes)
- Adjust scoring weights (40/30/20/10 → different ratios)
- Add more replenishment urgency levels
- Prioritize certain categories (SPF = health priority)
- Filter out products customer explicitly doesn't want

### **Medium Enhancements** (Logic Updates)
- Add bundle detection (suggest bundles of missing products)
- Add premium upsells (suggest premium version of owned products)
- Add quantity upsells ("Buy 2, get 1 free")
- Add category-based filtering (learn from customer behavior)

### **Hard Enhancements** (Architecture Changes)
- Add machine learning scoring (train on conversion data)
- Add collaborative filtering ("customers like you also bought")
- Add real-time personalization (track interactions, adjust recommendations)
- Add multi-customer batch processing
- Add LLM for natural language rationale generation

---

## 📚 Key Files

- **`src/cross_sell_upsell/data_utils.py`** - Data loading
- **`src/cross_sell_upsell/business_logic.py`** - Business rules
- **`src/cross_sell_upsell/nodes.py`** - LangGraph nodes
- **`src/cross_sell_upsell/workflow.py`** - Workflow definition
- **`src/cross_sell_upsell/state_utils.py`** - Debugging tools
- **`config.py`** - State schema and config
- **`run_cross_sell_orchestrator.py`** - Execution script

---

## 🎓 What You've Learned

1. **Orchestrator Architecture** - Nodes vs utilities separation
2. **State Design** - Multi-dimensional state for complex workflows
3. **Business Logic** - Rule-based scoring with multiple objectives
4. **Incremental Development** - Build utilities → nodes → workflow
5. **Testing Strategy** - Test utilities independently before nodes

---

## 🎉 Next Steps

1. **Test the Orchestrator** - Run it for different customers
2. **Review Output** - Check if recommendations make sense
3. **Adjust Scoring** - Tune weights based on business priorities
4. **Add Features** - Implement bundles, premium upsells, etc.
5. **Collect Feedback** - Use real customer data to improve

---

**The architecture is complete. Now you can experiment with different business logic, scoring approaches, and features - all without changing the core orchestration structure!**

*This is the power of orchestrator agents: the foundation enables endless possibilities.* 🚀

